In [ ]:
import customtkinter as ctk
import json
from tkinter import filedialog
import os # Ditambahkan untuk operasi path

# Konstanta untuk nama file default
DEFAULT_SAVE_FILENAME = "tree_data.json"

class Node:
    """
    Merepresentasikan sebuah simpul (node) dalam N-ary tree.
    Setiap node memiliki nilai, daftar anak, dan referensi ke induknya.
    """
    def __init__(self, value, parent=None):
        self.value = value
        self.children = []
        self.parent = parent

    def add_child(self, child_node):
        """Menambahkan child_node sebagai anak dari node ini."""
        child_node.parent = self
        self.children.append(child_node)

    def get_tree_representation(self):
        """Menghasilkan representasi string dari subtree yang berakar di node ini."""
        # Kondisi 'if not self:' dalam instance method akan selalu salah karena 'self' merujuk ke instance yang ada.
        # Jika node itu sendiri adalah None (misalnya, jika root adalah None dan kita mencoba memanggil ini padanya),
        # itu akan ditangani sebelum memanggil metode ini (misalnya, di N_aryTree.get_tree_string).
        # Jadi, kita bisa berasumsi 'self' adalah Node yang valid di sini.
        
        lines = [str(self.value)] 
        
        children_count = len(self.children)
        for i, child in enumerate(self.children):
            child._append_subtree_lines(lines, "", i == children_count - 1)
        return "\n".join(lines)

    def _append_subtree_lines(self, lines, prefix, is_last_child):
        lines.append(prefix + ("└── " if is_last_child else "├── ") + str(self.value))
        
        children_count = len(self.children)
        for i, child in enumerate(self.children):
            new_prefix = prefix + ("    " if is_last_child else "│   ")
            child._append_subtree_lines(lines, new_prefix, i == children_count - 1)

    def to_dict(self):
        return {
            "value": self.value,
            "children": [child.to_dict() for child in self.children]
        }

class N_aryTree:
    def __init__(self):
        self.root = None

    def add_root(self, value):
        if self.root is None:
            self.root = Node(value)
            return self.root
        else:
            print("Root sudah ada.") 
            return None

    def _find_node_recursive(self, current_node, target_value):
        if current_node is None:
            return None
        if current_node.value == target_value:
            return current_node
        for child in current_node.children:
            found_node = self._find_node_recursive(child, target_value)
            if found_node:
                return found_node
        return None

    def find_node(self, target_value):
        if not self.root:
            return None
        return self._find_node_recursive(self.root, target_value)

    def add_node(self, parent_value, child_value):
        if self.root is None:
            return None
        parent_node = self.find_node(parent_value)
        if parent_node:
            child_node = Node(child_value, parent=parent_node)
            parent_node.add_child(child_node)
            return child_node
        else:
            return None

    def add_leaf(self, parent_value, leaf_value):
        return self.add_node(parent_value, leaf_value)

    def get_tree_string(self):
        if self.root:
            return self.root.get_tree_representation()
        else:
            return "Tree kosong."

    def to_dict(self):
        if self.root:
            return self.root.to_dict()
        return None

    def save_to_file(self, filepath):
        tree_data = self.to_dict()
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(tree_data, f, indent=4)
            return True
        except IOError as e:
            print(f"Error saving tree to {filepath}: {e}")
            return False
        except Exception as e:
            print(f"An unexpected error occurred during saving: {e}")
            return False

    @classmethod
    def _build_tree_recursive(cls, node_dict_data, parent_node_instance=None):
        """
        Membangun node dan anak-anaknya secara rekursif dari data dictionary.
        """
        if not node_dict_data or 'value' not in node_dict_data:
            return None

        current_node = Node(node_dict_data['value'], parent=parent_node_instance)

        if 'children' in node_dict_data:
            for child_dict in node_dict_data['children']:
                child_node = cls._build_tree_recursive(child_dict, parent_node_instance=current_node)
                if child_node:
                    current_node.children.append(child_node) # add_child tidak perlu dipanggil karena parent sudah diatur
        return current_node

    @classmethod
    def load_from_file(cls, filepath):
        """Memuat tree dari file JSON."""
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                tree_data_root_dict = json.load(f) 
            
            if not tree_data_root_dict: 
                return cls() # Mengembalikan tree kosong jika file JSON kosong atau null

            new_tree = cls()
            new_tree.root = cls._build_tree_recursive(tree_data_root_dict, parent_node_instance=None)
            return new_tree
        except FileNotFoundError:
            print(f"Error: File tidak ditemukan di {filepath}")
            return None # Mengembalikan None jika file tidak ditemukan
        except json.JSONDecodeError:
            print(f"Error: Tidak dapat mendekode JSON dari {filepath}")
            return None # Mengembalikan None jika JSON tidak valid
        except Exception as e:
            print(f"Terjadi kesalahan tak terduga saat memuat: {e}")
            return None # Mengembalikan None untuk error lainnya

class App(ctk.CTk):
    def __init__(self): # tree_instance tidak lagi diperlukan sebagai argumen
        super().__init__()
        
        # Dapatkan direktori tempat skrip dijalankan
        try:
            self.script_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError: # __file__ tidak terdefinisi
            self.script_dir = os.getcwd() # Gunakan direktori kerja saat ini sebagai fallback

        self.default_filepath = os.path.join(self.script_dir, DEFAULT_SAVE_FILENAME)
        
        self.title("N-ary Tree GUI (CustomTkinter)")
        self.geometry("600x750") 

        ctk.set_appearance_mode("System") 
        ctk.set_default_color_theme("blue")

        # --- UI Elements ---
        self.root_frame = ctk.CTkFrame(self)
        self.root_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(self.root_frame, text="Nilai Root:").pack(side="left", padx=5)
        self.root_value_entry = ctk.CTkEntry(self.root_frame, placeholder_text="Masukkan nilai root")
        self.root_value_entry.pack(side="left", padx=5, expand=True, fill="x")
        self.add_root_button = ctk.CTkButton(self.root_frame, text="Tambah Root", command=self.gui_add_root)
        self.add_root_button.pack(side="left", padx=5)

        self.node_frame = ctk.CTkFrame(self)
        self.node_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(self.node_frame, text="Nilai Parent:").pack(side="left", padx=5)
        self.parent_value_entry = ctk.CTkEntry(self.node_frame, placeholder_text="Masukkan nilai parent")
        self.parent_value_entry.pack(side="left", padx=5, expand=True, fill="x")
        ctk.CTkLabel(self.node_frame, text="Nilai Child:").pack(side="left", padx=5)
        self.child_value_entry = ctk.CTkEntry(self.node_frame, placeholder_text="Masukkan nilai child")
        self.child_value_entry.pack(side="left", padx=5, expand=True, fill="x")
        self.add_node_button = ctk.CTkButton(self.node_frame, text="Tambah Node/Leaf", command=self.gui_add_node)
        self.add_node_button.pack(side="left", padx=5)

        self.file_ops_frame = ctk.CTkFrame(self)
        self.file_ops_frame.pack(pady=10, padx=10, fill="x")
        self.save_button = ctk.CTkButton(self.file_ops_frame, text="Simpan Tree", command=self.gui_save_tree)
        self.save_button.pack(side="left", padx=5)
        self.load_button = ctk.CTkButton(self.file_ops_frame, text="Muat Tree", command=self.gui_load_tree) 
        self.load_button.pack(side="left", padx=5)

        self.tree_display_textbox = ctk.CTkTextbox(self, height=300, width=580, wrap="none")
        self.tree_display_textbox.pack(pady=10, padx=10, fill="both", expand=True)
        self.tree_display_textbox.configure(state="disabled") 

        self.status_label = ctk.CTkLabel(self, text="Status: Siap", anchor="w") # Pastikan ini dibuat SEBELUM attempt_auto_load
        self.status_label.pack(pady=5, padx=10, fill="x")

        # Pindahkan pemuatan otomatis dan pembaruan tampilan setelah semua UI dibuat
        self.tree = self.attempt_auto_load() # Coba muat tree saat startup
        self.update_tree_display() # Perbarui tampilan setelah auto-load atau inisialisasi

    def attempt_auto_load(self):
        """Mencoba memuat tree dari file default saat startup."""
        if os.path.exists(self.default_filepath):
            loaded_tree = N_aryTree.load_from_file(self.default_filepath)
            if loaded_tree is not None: 
                self.update_status(f"Tree berhasil dimuat dari {DEFAULT_SAVE_FILENAME}")
                return loaded_tree
            else: 
                self.update_status(f"Gagal memuat tree dari {DEFAULT_SAVE_FILENAME}. File mungkin rusak atau tidak valid.")
                return N_aryTree() 
        else:
            self.update_status(f"File data default ({DEFAULT_SAVE_FILENAME}) tidak ditemukan. Memulai dengan tree kosong.")
            return N_aryTree() 


    def gui_add_root(self):
        value = self.root_value_entry.get()
        if not value:
            self.update_status("Nilai root tidak boleh kosong.")
            return
        if self.tree.root is not None:
            self.update_status(f"Root sudah ada: '{self.tree.root.value}'. Tidak bisa menambah root lagi.")
            return
        self.tree.add_root(value)
        self.update_status(f"Root '{value}' berhasil ditambahkan.")
        self.update_tree_display()
        self.root_value_entry.delete(0, "end")

    def gui_add_node(self):
        parent_value = self.parent_value_entry.get()
        child_value = self.child_value_entry.get()
        if not parent_value or not child_value:
            self.update_status("Nilai parent dan child tidak boleh kosong.")
            return
        if self.tree.root is None:
            self.update_status("Tree kosong. Tambahkan root terlebih dahulu.")
            return
        node_added = self.tree.add_node(parent_value, child_value)
        if node_added:
            self.update_status(f"Node '{child_value}' berhasil ditambahkan ke parent '{parent_value}'.")
            self.update_tree_display()
            self.parent_value_entry.delete(0, "end")
            self.child_value_entry.delete(0, "end")
        else:
            if not self.tree.find_node(parent_value):
                 self.update_status(f"Parent node '{parent_value}' tidak ditemukan.")
            else:
                self.update_status(f"Gagal menambahkan node '{child_value}'.")

    def gui_save_tree(self):
        if not self.tree.root:
            self.update_status("Tree kosong. Tidak ada yang bisa disimpan.")
            return
        # langsung simpan ke default_filepath
        if self.tree.save_to_file(self.default_filepath):
            self.update_status(f"Tree berhasil disimpan ke {DEFAULT_SAVE_FILENAME}")
        
        else:
            self.update_status(f"Gagal menyimpan tree ke {filepath}. Lihat konsol untuk detail.")

    def gui_load_tree(self):
        filepath = filedialog.askopenfilename(
            initialdir=self.script_dir, 
            defaultextension=".json",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")],
            title="Muat Tree Dari..."
        )
        if not filepath:
            self.update_status("Pemuatan dibatalkan.")
            return

        loaded_tree = N_aryTree.load_from_file(filepath)
        if loaded_tree is not None: 
            self.tree = loaded_tree 
            self.update_status(f"Tree berhasil dimuat dari {filepath}")
            self.update_tree_display()
        else: 
            self.update_status(f"Gagal memuat tree dari {filepath}. File mungkin rusak atau tidak valid.")


    def update_tree_display(self):
        self.tree_display_textbox.configure(state="normal")
        self.tree_display_textbox.delete("1.0", "end")
        tree_str = self.tree.get_tree_string()
        self.tree_display_textbox.insert("1.0", tree_str)
        self.tree_display_textbox.configure(state="disabled")

    def update_status(self, message):
        self.status_label.configure(text=f"Status: {message}")
        print(f"GUI Status: {message}")

if __name__ == "__main__":
    app = App() 
    app.mainloop()
